# MNIST Digit Recognition
## Introduction
This function of this Jupyter notebook is to explain how the python script [Source link to script](https://github.com/kbarry91/Emerging-Technologies/tree/master/4-MNIST%20Digit%20Recognition%20Script) works and to discuss its performance.
<br>

The aim of the script as discribed in the project brief was to develope a __Digit recognition script__: a _Python script that takes an image ﬁle containing a handwritten digit and identiﬁes the digit using a supervised learning algorithm and the MNIST dataset._


## Folder Structure 
On inital download of the project you will notice the directory only has one folder 'testImages' and a 'digitrec.py' script
<br>
__testImages:__ Contain images of different sizes and types to test the algorithm.<br>
__digitrec.py:__ A python script containing the source code
![initial](https://i.imgur.com/8nkerH3.png)
Once the program has been compiled and run another folder 'models' will be generated. 
![after](https://i.imgur.com/LTGgkJI.png)
__models:__ Contains saved neural network model to increase performace and decrease wait time 

## Navigating The program
The program has been build to provide simplicity to the user. That is it hides all the complicated workings from the user with a command line GUI that is simple to navigate.

### The main Menu 
The main menu allows the user to view a brief summary of the amount of images used to build and test the model. Aswell as this it also gives 4 options. An option is selected by entering a input into the command line
<br>
__1.Prepare Dataset and Image Recognition Model:__ Allows user to Build the Digit Recognition model.<br>
__2.Upload an Image to predict:__ Allows user to enter an image for the _testImages_ folder and make a prediction.<br>
__3.Use MNIST image to predict:__ Allows user to select an image from the MNIST training set to make a prediction.<br>
__4.Exit:__ Exits the program
![Menu](https://i.imgur.com/1MtWXvs.png)

### Prepare Dataset and Image Recognition model
Select option _1_ allows the user to Prepare the Dataset and to create the image recognition model. The user will be prompted with the option to select how many iterations will be used to train the model.Once the model is build it is saved in the directory as nn_model
<br>
The program will then display the following statistics
1. Summary info, The layers and shape of the model. (Achieved by the code _'model.summary()'_)
2. Build progress of the model training  (model.fit(image_train, label_train,
                                            batch_size=batch_size,
                                            epochs=epochs,
                                            verbose=1,
                                            validation_data=(image_test, label_test))
3. Build complete summary info including loss, accuracy and time taken

![build1](https://i.imgur.com/UwQ4DRS.png)
![build2](https://i.imgur.com/JEJ0wNK.png)

### Upload an Image to predict

### Use MNIST image to predict